In [1]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
INDEX = "MS8"
df = pd.read_csv(f"./data{INDEX}/merged_data.csv", parse_dates=["date"])

df = df.sort_values(["ticker", "date"])

features = ['d2c', 'shorts', 'trend']

In [ ]:
# Normalizziamo ogni serie per ticker (z-score)
df[features] = df.groupby('ticker')[features].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [ ]:
# Pivot per ottenere array: un titolo = una serie temporale multivariata
tickers = df['ticker'].unique()
dates = df['date'].unique()
time_series = []
for t in tickers:
    sub = df[df['ticker'] == t].set_index('date').reindex(dates).fillna(0)
    time_series.append(sub[features].values)

time_series = np.array(time_series)  # shape: (n_tickers, n_timepoints, n_features)

In [ ]:
n_clusters = 4  # numero di cluster
model = TimeSeriesKMeans(
    n_clusters=n_clusters,
    metric="dtw",
    max_iter=10,
    random_state=42,
    verbose=True
)
labels = model.fit_predict(time_series)